<a href="https://colab.research.google.com/github/hc417tw/118S_Course/blob/main/Prompt_Engineering_Exercise_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# I use Google Gemini for this coding exercise
# For prompt I ask Gemini to "I need a ReAct Python code to generate a study planner for student. Student will input each different deadline or exam time. Then I will need the python code to generate a study planner for the student in terms of importantance and urgency."
import datetime

class StudyReActAgent:
    def __init__(self):
        self.tasks = []

    def add_task(self, name, days_left, weight):
        # weight is 1-10 (Importance)
        self.tasks.append({"name": name, "days": days_left, "weight": weight})

    def reason_and_plan(self):
        print("--- 🧠 Agent Reasoning Phase ---")
        planned_schedule = []

        for task in self.tasks:
            # THOUGHT: The agent evaluates the task
            print(f"Thought: Analyzing '{task['name']}'. Due in {task['days']} days. Weight: {task['weight']}/10.")

            # ACTION: Categorize based on Eisenhower Matrix logic
            if task['days'] <= 2 and task['weight'] >= 7:
                category = "🔥 DO FIRST (Urgent & Important)"
                priority = 1
            elif task['weight'] >= 7:
                category = "📅 SCHEDULE (Important, Not Urgent)"
                priority = 2
            elif task['days'] <= 2:
                category = "⚡ DELEGATE/QUICK WIN (Urgent, Not Important)"
                priority = 3
            else:
                category = "🗑️ ELIMINATE/POSTPONE (Low Priority)"
                priority = 4

            # OBSERVATION: Store the result
            print(f"Action: Categorized as {category}")
            planned_schedule.append({**task, "category": category, "priority": priority})

        # Final Sort
        planned_schedule.sort(key=lambda x: x['priority'])
        return planned_schedule

def run_planner():
    agent = StudyReActAgent()
    print("--- 📚 AI Study Planner (ReAct Style) ---")

    while True:
        name = input("\nExam/Assignment Name (or 'done'): ")
        if name.lower() == 'done': break

        days = int(input(f"Days until '{name}': "))
        weight = int(input(f"Importance (1-10, where 10 is a Final Exam): "))
        agent.add_task(name, days, weight)

    # Execute ReAct Loop
    final_plan = agent.reason_and_plan()

    print("\n--- 🗓️ YOUR FINAL STUDY PLAN ---")
    for item in final_plan:
        print(f"[{item['category']}] {item['name']} - Focus Level: {item['weight']}/10")

run_planner()

--- 📚 AI Study Planner (ReAct Style) ---

Exam/Assignment Name (or 'done'): BUS3-189
Days until 'BUS3-189': 20
Importance (1-10, where 10 is a Final Exam): 10

Exam/Assignment Name (or 'done'): BUS4-118S
Days until 'BUS4-118S': 35
Importance (1-10, where 10 is a Final Exam): 7

Exam/Assignment Name (or 'done'): done
--- 🧠 Agent Reasoning Phase ---
Thought: Analyzing 'BUS3-189'. Due in 20 days. Weight: 10/10.
Action: Categorized as 📅 SCHEDULE (Important, Not Urgent)
Thought: Analyzing 'BUS4-118S'. Due in 35 days. Weight: 7/10.
Action: Categorized as 📅 SCHEDULE (Important, Not Urgent)

--- 🗓️ YOUR FINAL STUDY PLAN ---
[📅 SCHEDULE (Important, Not Urgent)] BUS3-189 - Focus Level: 10/10
[📅 SCHEDULE (Important, Not Urgent)] BUS4-118S - Focus Level: 7/10


In [4]:
# To improve from the last one, I further on ask Gemini to ask user for the date of exam or deadline, and calculate the best and suitable study plan for the student. Instead of just asking on importance and urgency.
import datetime
import re

# ==========================================
# 1. DEFINE THE TOOLS (The Agent's actions)
# ==========================================
def tool_calculate_days(date_str):
    """Calculates days from today to the target exam date (YYYY-MM-DD)."""
    try:
        target = datetime.datetime.strptime(date_str.strip(), "%Y-%m-%d").date()
        today = datetime.date.today()
        return str((target - today).days)
    except Exception as e:
        return f"Error parsing date: {e}"

def tool_generate_habit(days_left_str):
    """Generates a daily study habit based strictly on the days left."""
    try:
        days = int(days_left_str)
        if days < 0:
            return "Exam has passed. Take a break!"
        elif days <= 7:
            return "The Sprint: 2 hours of timed mock exams + 30 mins reviewing weak points."
        elif days <= 30:
            return "The Jog: 1 hour of active recall (flashcards) + 1 hour of practice problems."
        else:
            return "The Marathon: 30 mins reading new material + 15 mins spaced repetition review."
    except Exception:
        return "Error: Input must be a number."

# Register tools in a dictionary so the Agent can call them by name
tools = {
    "CalculateDays": tool_calculate_days,
    "GenerateHabit": tool_generate_habit
}

# ==========================================
# 2. THE REACT AGENT LOOP
# ==========================================
def run_react_agent(subject, exam_date):
    print("--- 🤖 ReAct Agent Initialized ---")
    print(f"Goal: Create a study plan for '{subject}' on {exam_date}\n")

    # MOCKING THE LLM:
    # In a real app (like LangChain), an LLM generates these strings dynamically.
    # For Colab testing without an API key, we simulate the LLM's text generation.
    llm_generated_responses = [
        f"Thought: I need to find out exactly how many days are left until the {subject} exam on {exam_date}.\nAction: CalculateDays[{exam_date}]",
        "Thought: Now that I have the exact days left, I need to generate a specific daily study habit for this timeframe.\nAction: GenerateHabit[{last_observation}]",
        "Thought: I have the generated study habit. I can now compile the final answer for the student.\nFinal Answer: Based on the timeline of {days_left} days, your daily study habit for {subject} should be: {final_habit}"
    ]

    # The Execution Engine (Memory and Loop)
    memory = {}

    for step, text_block in enumerate(llm_generated_responses):
        # 1. Inject context into the LLM's generated text (simulating context window)
        text_block = text_block.format(
            subject=subject,
            exam_date=exam_date,
            last_observation=memory.get("last_obs", ""),
            days_left=memory.get("days_left", ""),
            final_habit=memory.get("last_obs", "")
        )

        # 2. Print the Thought
        thought_line = text_block.split('\n')[0]
        print(thought_line)

        # 3. Parse for a Final Answer to break the loop
        if "Final Answer:" in text_block:
            final_answer = text_block.split("Final Answer:")[1].strip()
            print(f"\n✅ Final Answer: {final_answer}\n")
            break

        # 4. Parse for Actions using Regex
        match = re.search(r"Action:\s*(\w+)\[(.*?)\]", text_block)
        if match:
            action_name = match.group(1)
            action_input = match.group(2)
            print(f"Action: {action_name}[{action_input}]")

            # 5. Execute the Tool and record the Observation
            if action_name in tools:
                observation = tools[action_name](action_input)
                print(f"Observation: {observation}\n")

                # Update memory
                memory["last_obs"] = observation
                if action_name == "CalculateDays":
                    memory["days_left"] = observation
            else:
                print(f"Observation: Tool '{action_name}' not found.\n")

# ==========================================
# 3. USER INTERFACE
# ==========================================
def start():
    print("Welcome to the ReAct Study Planner.")
    subj = input("Enter subject (e.g., Biology): ").strip()
    date_in = input("Enter exam date (YYYY-MM-DD): ").strip()
    print("\n" + "="*50 + "\n")

    run_react_agent(subj, date_in)

# Run the program
start()

Welcome to the ReAct Study Planner.
Enter subject (e.g., Biology): Agentic AI
Enter exam date (YYYY-MM-DD): 2026-03-05


--- 🤖 ReAct Agent Initialized ---
Goal: Create a study plan for 'Agentic AI' on 2026-03-05

Thought: I need to find out exactly how many days are left until the Agentic AI exam on 2026-03-05.
Action: CalculateDays[2026-03-05]
Observation: 7

Thought: Now that I have the exact days left, I need to generate a specific daily study habit for this timeframe.
Action: GenerateHabit[7]
Observation: The Sprint: 2 hours of timed mock exams + 30 mins reviewing weak points.

Thought: I have the generated study habit. I can now compile the final answer for the student.

✅ Final Answer: Based on the timeline of 7 days, your daily study habit for Agentic AI should be: The Sprint: 2 hours of timed mock exams + 30 mins reviewing weak points.

